Here we load our tools: Pandas for data, NumPy for math, and our two agents.

In [1]:
import sys
sys.path.append("../src")

import pandas as pd
import numpy as np

from strategy_agent import StrategyAgent
from risk_agent import RiskAgent


We’ll test on Apple’s enriched dataset again.

In [2]:
data = pd.read_csv("../data/AAPL_enriched.csv", index_col=0, parse_dates=True)


This computes CAGR, volatility, Sharpe, and drawdown — the KPIs RiskAgent needs.

In [3]:
def compute_metrics(df):
    strat_curve = (1 + df["Strategy_Return"]).cumprod()
    years = (df.index[-1] - df.index[0]).days / 365.25
    cagr = strat_curve.iloc[-1]**(1/years) - 1
    vol = df["Strategy_Return"].std() * np.sqrt(252)
    sharpe = (df["Strategy_Return"].mean() * 252) / (df["Strategy_Return"].std() * np.sqrt(252))
    roll_max = strat_curve.cummax()
    dd = (strat_curve / roll_max - 1).min()
    return {"CAGR": cagr, "Volatility": vol, "Sharpe": sharpe, "MaxDD": dd}


The StrategyAgent generates signals/returns for SMA, RSI, and Bollinger.

In [4]:
agent = StrategyAgent(data)

sma_res = agent.sma_crossover()
rsi_res = agent.rsi_strategy()
bb_res = agent.bollinger_breakout()


We compute metrics for each strategy separately so RiskAgent can judge them.

In [5]:
strategies = {
    "SMA Crossover": sma_res,
    "RSI": rsi_res,
    "Bollinger": bb_res
}

metrics_results = {name: compute_metrics(df) for name, df in strategies.items()}
pd.DataFrame(metrics_results).T


,CAGR,Volatility,Sharpe,MaxDD
SMA Crossover,-0.121856,0.041243,-3.175536,-0.026330
RSI,-0.126123,0.082338,-1.620564,-0.043275
Bollinger,0.139849,0.044243,3.024177,-0.007036


RiskAgent reviews each strategy’s metrics, applies thresholds, and flags pass/fail with notes.

In [6]:
risk = RiskAgent(sharpe_min=1.0, max_dd=-0.25, vol_max=0.3)

risk_assessments = {}
for name, m in metrics_results.items():
    risk_assessments[name] = risk.assess(m)

risk_assessments


{'SMA Crossover': {'Decision': 'FAIL',
  'Notes': ['Sharpe too low (-3.18 < 1.0)']},
 'RSI': {'Decision': 'FAIL', 'Notes': ['Sharpe too low (-1.62 < 1.0)']},
 'Bollinger': {'Decision': 'PASS', 'Notes': ['All risk checks passed']}}

We combine everything into a single DataFrame — one row per strategy, with both numbers and pass/fail notes.

In [7]:
final_results = []
for name, m in metrics_results.items():
    assessment = risk_assessments[name]
    final_results.append({
        "Strategy": name,
        **m,
        "Decision": assessment["Decision"],
        "Notes": "; ".join(assessment["Notes"])
    })

final_df = pd.DataFrame(final_results)
final_df


,Strategy,CAGR,Volatility,Sharpe,MaxDD,Decision,Notes
0,SMA Crossover,-0.121856,0.041243,-3.175536,-0.026330,FAIL,Sharpe too low (-3.18 < 1.0)
1,RSI,-0.126123,0.082338,-1.620564,-0.043275,FAIL,Sharpe too low (-1.62 < 1.0)
2,Bollinger,0.139849,0.044243,3.024177,-0.007036,PASS,All risk checks passed


Save results to /logs/Day13_strategy_risk_results.csv so you have an audit trail.

In [8]:
import os
from datetime import datetime

os.makedirs("../logs", exist_ok=True)
final_df.to_csv("../logs/Day13_strategy_risk_results.csv", index=False)

with open("../logs/Day13_summary.txt", "w") as f:
    f.write(f"Day 13 run ({datetime.today().date()})\n")
    f.write(final_df.to_string())
